In [1]:
import pandas as pd
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split

In [2]:
import lightgbm as lgb

# LightGBM model and parameters

In [3]:
params = {}
params['learning_rate'] = 0.09043007325486686
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 45
params['min_data'] = 50
params['max_depth'] = 10

# Hyper parameter tuning from kaggle kernel:
{'boosting_type': 0,
 'colsample_by_tree': 0.7073741758938382,
 'gdbt_subsample': 0.576055003450302,
 'is_unbalance': 0,
 'learning_rate': 0.09043007325486686,
 'min_child_samples': 270.0,
 'num_leaves': 45.0,
 'reg_alpha': 0.14138716535199491,
 'reg_lambda': 0.2889595293526205,
 'subsample_for_bin': 200000.0}



In [4]:
# Prepare datasets
X = pd.read_csv('result_data/train_app_prev_b_merged_corr.csv')
test_data = pd.read_csv('result_data/test_app_prev_b_merged_corr.csv')
Y  = X['TARGET']
X  = X.drop(columns=['TARGET'],axis=1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
d_train = lgb.Dataset(X, label=Y)

In [6]:
clf = lgb.train(params, d_train, 200)

In [7]:
y_pred=clf.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
type(y_test.values)
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [ ]:
auc(fpr, tpr)

# Results

1. app_precessed data  :   74.23      test_roc: 70
2. train_app_bureau_merged: 0.7298677  test_roc: 0.721
3. train_app_bu_mer_corr_drp: 0.73263626999423  test_roc: 0.724
4. Same: Leaves 60 : 0.7371016183398422
5. Same: + LR : 0.09043007325486686 + leaves 45: 0.7409575603828592
6. Same + 270 estimators : 0.7591253403483376    test_roc: 0.742
7. Same + 230 estimators: 0.7601499389379056     test_roc:
8. Same + 200 : 0.7605678355570721
9. Domain Features + same + no_crr_drp : 0.7736541538631981
10. Domain Features + same + crr_drp : 0.76

# Predict test data


In [8]:
predict_data = clf.predict(test_data)


In [9]:
df_pred = pd.DataFrame(predict_data)
df_pred['SK_ID_CURR'] =test_data['SK_ID_CURR']
df_pred.head()

,0,SK_ID_CURR
0,0.028906,100001
1,0.121161,100005
2,0.019349,100013
3,0.021183,100028
4,0.161876,100038


In [10]:
df_pred.set_index('SK_ID_CURR',inplace=True)
df_pred.rename(columns={0:'TARGET'},inplace=True)

In [11]:
df_pred.head()

,TARGET
SK_ID_CURR,
100001,0.028906
100005,0.121161
100013,0.019349
100028,0.021183
100038,0.161876


In [12]:
df_pred.to_csv('test output/pred_bureu_prev_corr_lbm_fourth__hyper_200_est.csv',index=True)

# XGBoost Model

In [ ]:
from xgboost import XGBClassifier
import numpy as np

In [ ]:
# Prepare datasets
X = pd.read_csv('result_data/train_app_bureau_merged.csv')
test_data = pd.read_csv('result_data/test_app_bureau_merged.csv')
Y  = X['TARGET']
X  = X.drop(columns=['TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Create a xgboost model
model = XGBClassifier(learning_rate=0.09043007325486686,n_jobs=-1,n_estimators=200)
model.fit(X_train,y_train)

In [ ]:
y_predict = model.predict_proba(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test.values,y_pred)

In [ ]:
auc(fpr, tpr)

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_roc(y_test.values, y_predict)
#plt.show()